In [60]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import load_img, img_to_array, smart_resize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [61]:
# Load the dataset
data = pd.read_csv('./data/fashion.csv')

# Extract the image paths and gender labels
image_paths = data.apply(lambda row: f"./data/{row['Category']}/{row['Gender']}/Images/images_with_product_ids/{row['Image']}", axis=1).tolist()
labels = data['Gender']

In [63]:
# Define the desired image size
image_width, image_height = 180, 240

In [64]:
# Load and preprocess the images
X = []
for image_path in image_paths:
    image = load_img(image_path)
    image = smart_resize(image, (image_width, image_height))
    image = img_to_array(image) / 255.0
    X.append(image)
X = np.array(X)

In [65]:
# Encode the gender labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)

In [67]:
# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [68]:
# Create the neural network model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_width, image_height, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(4, activation='softmax'))

In [69]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [70]:
# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.2)

Epoch 1/10
59/59 [==============================] - 83s 1s/step - loss: 1.6576 - accuracy: 0.6955 - val_loss: 0.6230 - val_accuracy: 0.6968
Epoch 2/10
59/59 [==============================] - 84s 1s/step - loss: 0.3359 - accuracy: 0.8548 - val_loss: 0.4211 - val_accuracy: 0.8323
Epoch 3/10
59/59 [==============================] - 83s 1s/step - loss: 0.1962 - accuracy: 0.9322 - val_loss: 0.3758 - val_accuracy: 0.8602
Epoch 4/10
59/59 [==============================] - 84s 1s/step - loss: 0.1256 - accuracy: 0.9570 - val_loss: 0.4540 - val_accuracy: 0.8559
Epoch 5/10
59/59 [==============================] - 84s 1s/step - loss: 0.1276 - accuracy: 0.9554 - val_loss: 0.4898 - val_accuracy: 0.8452
Epoch 6/10
59/59 [==============================] - 83s 1s/step - loss: 0.0987 - accuracy: 0.9726 - val_loss: 0.3952 - val_accuracy: 0.8667
Epoch 7/10
59/59 [==============================] - 83s 1s/step - loss: 0.0462 - accuracy: 0.9882 - val_loss: 0.4467 - val_accuracy: 0.8860
Epoch 8/10
59/59 [==

In [71]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

19/19 [==============================] - 5s 281ms/step - loss: 0.7678 - accuracy: 0.8711
Test Loss: 0.767772376537323
Test Accuracy: 0.8711340427398682


In [107]:
# Make predictions
predictions = model.predict(X_test)
predicted_genders = label_encoder.inverse_transform(np.argmax(predictions, axis=1))

19/19 [==============================] - 7s 340ms/step


In [108]:
## Testing the Image on some random image
from tensorflow.keras.preprocessing.image import load_img, img_to_array, smart_resize

image_path = "./download2.webp"  # Replace with the path to your image
image = load_img(image_path)
image = smart_resize(image, (image_width, image_height))
image = img_to_array(image) / 255.0
image = np.expand_dims(image, axis=0)

In [109]:
prediction = model.predict(image)
predicted_gender = label_encoder.inverse_transform(np.argmax(prediction, axis=1))

1/1 [==============================] - 0s 44ms/step


In [110]:
# Make predictions
prediction = model.predict(image)
predicted_gender = label_encoder.inverse_transform(np.argmax(prediction, axis=1))
softmax_probabilities = prediction[0]  # Assuming prediction shape is (1, num_classes)
    
if predicted_gender in ["Boys", "Girls"]:
    product_type = "Apparel"
elif predicted_gender in ["Men", "Women"]:
    product_type = "Footwear"
else:
    product_type = "Unknown"

print("Product Type:", product_type)
print("Gender:", predicted_gender[0])

1/1 [==============================] - 0s 44ms/step
Product Type: Apparel
Gender: Girls
